In [ ]:
import numpy as np

class Project:
    def __init__(self, project_type, participated_in_past_rounds, elected_in_past_rounds):
        self.project_type = project_type
        self.participated_in_past_rounds = participated_in_past_rounds
        self.elected_in_past_rounds = elected_in_past_rounds

# Example usage
projects = [Project(np.random.choice(['type1', 'type2']), 
                    np.random.choice([True, False]), 
                    np.random.choice([True, False])) for _ in range(num_projects)]


In [ ]:
class Voter:
    def __init__(self, strategy):
        self.strategy = strategy
    
    def vote(self, projects):
        if self.strategy == 'few':
            a=10
        elif self.strategy == 'most':
            a=15
        elif self.strategy == 'equally':
            a=25
        elif self.strategy == 'mass_few':
            a=30

# Example usage
voters = [Voter(np.random.choice(['few', 'most', 'equally', 'mass_few'])) for _ in range(num_voters)]


In [ ]:
def mean(votes):
    return np.mean(votes, axis=0)

def median_with_moving_phantoms(votes):
    # Implement moving phantoms logic
    pass

def quadratic_mean(votes, k):
    return np.power(np.mean(np.power(votes, k), axis=0), 1/k)

def quadratic_median_with_moving_phantoms(votes, k):
    # Implement moving phantoms with quadratic median logic
    pass

# Example usage
results_mean = mean(votes_matrix)
results_median_phantoms = median_with_moving_phantoms(votes_matrix)
results_quadratic_mean = quadratic_mean(votes_matrix, k=0.5)
results_quadratic_median_phantoms = quadratic_median_with_moving_phantoms(votes_matrix, k=0.5)


In [ ]:
def evaluate_solution(solution):
    # Logic to evaluate the solution (e.g., resistance to manipulation, representativeness)
    pass

def robustness_check(solution, perturbations):
    original_solution = solution
    variances = []
    for perturb in perturbations:
        perturbed_solution = perturb
        variance = np.linalg.norm(original_solution - perturbed_solution, ord=1)
        variances.append(variance)
    return np.mean(variances)

# Example usage
average_score = np.mean([evaluate_solution(run_simulation()) for _ in range(num_instances)])
robustness_score = robustness_check(solution, perturbations)


In [ ]:
def run_simulation(num_projects, num_voters, alpha):
    votes_matrix = np.random.dirichlet(alpha, num_voters)
    # Implement the rest of the simulation logic

# Example usage
alpha = [1] * num_projects
num_projects = 10
num_voters = 100
results = run_simulation(num_projects, num_voters, alpha)


In [ ]:
def compare_parameters(parameter_combinations):
    results = []
    for params in parameter_combinations:
        scores = []
        for _ in range(num_instances):
            score = evaluate_solution(run_simulation(params['num_projects'], params['num_voters'], params['alpha']))
            scores.append(score)
        results.append({
            'params': params,
            'average_score': np.mean(scores),
            'robustness_score': robustness_check(solution, perturbations)
        })
    return results

# Example usage
parameter_combinations = [{'num_projects': 10, 'num_voters': 100, 'alpha': [1]*10}]
results_comparison = compare_parameters(parameter_combinations)
